In [3]:
import tensorflow as td
from tensorflow import keras
import numpy as np

# PREPROCESSING DATA

# getting data
data = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = data.load_data(num_words=10000)

word_index = data.get_word_index()

# marking the words that we don't recognise and other functional like pad
# word_index -> dictionary of tuples of keys and values to code the words
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# setting the data to be the fixed length (we need to know how many input neurons are there [in this case fixed 250 characters])
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=word_index["<PAD>"], padding="post", maxlen=250)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=word_index["<PAD>"], padding="post", maxlen=250)

# decoding the integers into words based on a dictionary of keys and values (reverse_word_index)
def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

print(len(test_data[0]), len(test_data[1]))

# DEFINING MODEL

model = keras.Sequential()
model.add(keras.layers.Embedding(10000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

68 260
